<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_09/blob/Test-2.4v2/Coursework_question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Computing & Numerical Methods Coursework Group 9
By: Ching Yau Chan, Hassan Alhamdani, Jiongjie Chen, Lucas So and Oyinmiebi Youdeowei

# Test 1

# Test 1.3

# Test 1.4

# Test 1.5

# Test 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation

end_time = float(input("How long should the model last for, in seconds  "))
time_interval = float(input("The time interval, in seconds   "))
length = float(input("Length of model, in metres   "))
length_interval = float(input("The length interval, in metres   "))
speed = float(input("Speed of river flow, in m/s   "))
initial_concentration = str(input("Name of the .csv file   "))

def concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration):
    distance = np.array([])
    for i in range(int(length / length_interval + 1)):
        distance = np.append(distance, length_interval * i)

    df = pd.read_csv(initial_concentration, encoding='latin1')

    x_data = df["Distance (m)"].values
    C_data = df["Concentration (µg/m_ )"].values

    initial_conditions = np.interp(distance, x_data, C_data, left=0.0, right=0.0)

    fig, ax = plt.subplots()

    max_C0 = np.max(initial_conditions)

    A_value = 1 / time_interval + speed / length_interval
    B_value = speed / length_interval

    A_array = np.array([])
    B_array = np.array([])

    for i in range(int(length / length_interval + 1)):
        A_array = np.append(A_array, A_value)
        B_array = np.append(B_array, B_value)

    num_frames = (int(end_time / time_interval)+1)

    current_initial_conditions = initial_conditions.copy()

    all_concentrations = []

    for k in range(num_frames):
        if k == 0:
            all_concentrations.append(initial_conditions.copy())
        else:
            concentration_present = np.zeros(int(length / length_interval + 1))
            concentration_present[0] = df.iloc[0, 1]

            for i in range(1, int(length / length_interval + 1)):
                concentration_present[i] = (current_initial_conditions[i] / time_interval + B_array[i] * concentration_present[i - 1]) / A_array[i]

            current_initial_conditions = concentration_present.copy()
            all_concentrations.append(concentration_present.copy())

    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150
    plt.ioff()

    def animate(f):

        plt.cla()

        concentration = all_concentrations[f]

        plt.plot(distance, concentration)

        plt.xlim(0, length)
        plt.ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)
        plt.xlabel("Distance (m)")
        plt.ylabel("Concentration (μg/m³)")
        plt.title(f"Test 2: Concentration at Time = {f * time_interval:.1f} s")
        plt.grid(True)

    anim = matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)
    return anim

anim = concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration)
anim

# Test 2.3

# Test 2.4

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt# mathplot is used to make a 'figure' where the ax graph can be mapped onto.
import matplotlib.animation

# These are the inputs that act as parameters that allow the user to specify model domain, resolution, and boundary conditions at the edge of the domain
end_time = float(input("How long should the model last for, in seconds  "))
time_interval = float(input("The time interval, in seconds   "))
length = float(input("Length of model, in metres   "))
length_interval = float(input("The length interval, in metres   "))
speed = float(input("Speed of river flow, in m/s   "))
initial_concentration = str(input("Name of the .csv file   "))
decay_constant = float(input("Decay constant for exponential decay (e.g., 0.01), enter 0 for no decay   ")) # addition

# Create a one-dimensional spatial grid
def concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration, decay_constant): # addition
    distance = np.array([])# Creates an empty array to hold 'distance' values
    for i in range(int(length / length_interval + 1)):
        distance = np.append(distance, length_interval * i)

    # Read the initial condition data in the CSV file，using "latin1" to make sure the correct format of the file
    df = pd.read_csv(initial_concentration, encoding='latin1')

    # Get the datas from the CSV file
    x_data = df["Distance (m)"].values
    C_data = df["Concentration (µg/m_ )"].values

    # Interpolate the concentration onto the grid, Set the concentration over measurement range to 0
    initial_conditions = np.interp(distance, x_data, C_data, left=0.0, right=0.0)

    # Extract the initial pollutant concentration at position x = 0 from the CSV file and save it as C0_initial
    C0_initial = df.iloc[0, 1] # addition

    # Create coordinate axes
    fig, ax = plt.subplots()

    # X-axis range
    ax.set_xlim(0, length)

    # Y-axis range Multiply by 1.1 to avoid the curve over the frame
    max_C0 = np.max(initial_conditions)
    ax.set_ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)

    # Set axis labels
    ax.set_xlabel("Distance (m)")
    ax.set_ylabel("Concentration (μg/m^3)")

    # Calculate the coefficients A and B according to the equation
    A_value = 1 / time_interval + speed / length_interval
    B_value = speed / length_interval

    # Create an array of coefficients A and B
    A_array = np.array([])
    B_array = np.array([])

    for i in range(int(length / length_interval + 1)):
        A_array = np.append(A_array, A_value)
        B_array = np.append(B_array, B_value)

    # Calculate the total number of frames
    num_frames = (int(end_time / time_interval)+1)

    # Record the current concentration distribution
    current_initial_conditions = initial_conditions.copy()

    all_concentrations = []

    # Calculate the concentration for each time and store it in all_concentrations
    for k in range(num_frames):
        if k == 0:
            all_concentrations.append(initial_conditions.copy())
        else:
            concentration_present = np.zeros(int(length / length_interval + 1))
            current_time = k * time_interval # addition
            concentration_present[0] = C0_initial * np.exp(-decay_constant * current_time) # addition
            # concentration_present[0] = df.iloc[0, 1] -> deleted010

            for i in range(1, int(length / length_interval + 1)):
                concentration_present[i] = (current_initial_conditions[i] / time_interval + B_array[i] * concentration_present[i - 1]) / A_array[i]

            current_initial_conditions = concentration_present.copy()
            all_concentrations.append(concentration_present.copy())

    # Set the animation parameters of Matplotlib
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150
    plt.ioff()# Turn off interactions to avoid conflicts between animations and interactions

    def animate(f):

        plt.cla()

        concentration = all_concentrations[f]# Find the concentration distribution corresponding to Frame f

        plt.plot(distance, concentration)# Draw the concentration-distance curve

        # Reset the coordinate axis range and labels, titles
        plt.xlim(0, length)
        plt.ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)
        plt.xlabel("Distance (m)")
        plt.ylabel("Concentration (μg/m³)")
        plt.title(f"Test 2.4: Concentration at Time = {f * time_interval:.1f} s (decay = {decay_constant})") # addition
        plt.grid(True)

    anim = matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)# Create animations
    return anim

anim = concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration, decay_constant) # addition
anim# Play and display the animation

# Test 2.5

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt# mathplot is used to make a 'figure' where the ax graph can be mapped onto.
import matplotlib.animation

# These are the inputs that act as parameters that allow the user to specify model domain, resolution, and boundary conditions at the edge of the domain
end_time = float(input("How long should the model last for, in seconds  "))
time_interval = float(input("The time interval, in seconds   "))
length = float(input("Length of model, in metres   "))
length_interval = float(input("The length interval, in metres   "))
speed = float(input("Speed of river flow, in m/s   "))
random_perturbation = float(input("The minimum and maximum of random perturbation of the stream, in percentages   "))
initial_concentration = str(input("Name of the .csv file   "))

# Create a one-dimensional spatial grid
def concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration):
    distance = np.array([])# Creates an empty array to hold 'distance' values
    for i in range(int(length / length_interval + 1)):
        distance = np.append(distance, length_interval * i)

    # Read the initial condition data in the CSV file，using "latin1" to make sure the correct format of the file
    df = pd.read_csv(initial_concentration, encoding='latin1')

    # Get the datas from the CSV file
    x_data = df["Distance (m)"].values
    C_data = df["Concentration (µg/m_ )"].values

    # Interpolate the concentration onto the grid, Set the concentration over measurement range to 0
    initial_conditions = np.interp(distance, x_data, C_data, left=0.0, right=0.0)

    # Create coordinate axes
    fig, ax = plt.subplots()

    max_C0 = np.max(initial_conditions)

    # Calculate the total number of frames
    num_frames = (int(end_time / time_interval)+1)

    # Record the current concentration distribution
    current_initial_conditions = initial_conditions.copy()

    all_concentrations = []

    # Calculate the concentration for each time and store it in all_concentrations
    for k in range(num_frames):
        if k == 0:
            all_concentrations.append(initial_conditions.copy())
        else:
            concentration_present = np.zeros(int(length / length_interval + 1))
            concentration_present[0] = df.iloc[0, 1]

            # Emulating variable speed with a factor between 1 +- random perturbation,
            random_variable = np.random.random((int(length / length_interval+1)))
            random_speed = (1 - random_perturbation/100 + random_perturbation/50 * random_variable) * speed

            # Random speed affects A and B values
            A_value = 1/time_interval + random_speed / length_interval
            B_value = random_speed / length_interval

            for i in range(1, int(length / length_interval + 1)):
                concentration_present[i] = (current_initial_conditions[i] / time_interval + B_value[i] * concentration_present[i - 1]) / A_value[i]

            current_initial_conditions = concentration_present.copy()
            all_concentrations.append(concentration_present.copy())

    # Set the animation parameters of Matplotlib
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 150
    plt.ioff()# Turn off interactions to avoid conflicts between animations and interactions

    def animate(f):

        plt.cla()

        concentration = all_concentrations[f]# Find the concentration distribution corresponding to Frame f

        plt.plot(distance, concentration)# Draw the concentration-distance curve

        # Reset the coordinate axis range and labels, titles
        plt.xlim(0, length)
        plt.ylim(0, max_C0 * 1.1 if max_C0 > 0 else 1.0)
        plt.xlabel("Distance (m)")
        plt.ylabel("Concentration (μg/m³)")
        plt.title(f"Test 2.5: Concentration at Time = {f * time_interval:.1f} s (Velocity perturbation: ±{random_perturbation}%)")
        plt.grid(True)

    anim = matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)# Create animations
    return anim

anim = concentration_from_csv(end_time, time_interval, length, length_interval, speed, initial_concentration)
anim# Play and display the animation

The random perturbation on speed is most visible when there is a consistent concentration decrease over distance (e.g. t = 100s) instead of stagnation (e.g. t = 270s) in concentration. Interestingly, at the early stages of the graph, the bumpiness of the curve is more visible compared to the late stages of the graph, mostly because the initial concentration does not decrease over distance. Compared to the graph for consistent speed, the lines are bumpy and not smooth due to slight increase or decrease in rate of change of concentration from fluctuating speed.